## WORKSHOP DE SCRAPING

Vamos a scrapear la web de Carrefour en busqueda de cervezas

___

In [1]:
from selenium import webdriver
from time import sleep
from bs4 import BeautifulSoup as BS
import requests

In [2]:
driver = webdriver.Chrome(executable_path='../../chrDriver/chromedriver.exe')

url = 'https://supermercado.carrefour.com.ar/catalogsearch/result/?q=cerveza'
driver.get(url)

js_scroll_dar_posicion = '''
    fin_de_pantalla = document.body.scrollHeight
    window.scrollTo(0, fin_de_pantalla)
    return fin_de_pantalla
'''
js_click_boton ="""
    boton = document.querySelector('[class="ver-mas-productos btn meanbee-infinitescroll-button"]')
                               if (boton){
                                       boton.click()
                                    } else{
                                        return "Fin Paginador"
                                    }
"""
pos_actual = 0
pos_siguiente = driver.execute_script(js_scroll_dar_posicion)
sleep(2)

while pos_actual != pos_siguiente:
    pos_siguiente = pos_actual
    pos_actual = driver.execute_script(js_scroll_dar_posicion)
    sleep(2)
    click_boton = driver.execute_script(js_click_boton)

___

Una vez desplegamos todo el html de la pagina procedemos a scrapear la informacion requerida

___

In [3]:
html = driver.execute_script("return document.documentElement.outerHTML")
dom = BS(html, features = 'html.parser')
cervezas = dom.find_all(class_ = 'producto-info')

In [4]:
links = [cerveza.a['href'] for cerveza in cervezas]

descripcion = [cerveza.a['title'] for cerveza in cervezas]

precio_reg = [cerveza.find(class_='regular-price').text.replace('Precio regular:', '').strip() for cerveza in cervezas]

precio_lt = [cerveza.find(class_='precio-medida').text.strip() for cerveza in cervezas]

marca = [cerveza.find(class_='brand truncate').text.strip() for cerveza in cervezas]


In [6]:
#como no todos los productos tienen promo hago un manejo de excepciones
promo = []
desc_promo = []
for cerveza in cervezas:
    try:
        promo.append(cerveza.find(class_='precio-numero').text)
        desc_promo.append(cerveza.find(class_='precio-texto').text.replace('\n', '').replace('   ', ''))
    except:
        promo.append('Null')
        desc_promo.append('Null')

___

Uno de los datos requeridos solo se encuentra al desplegar el modal de cada producto

Esto va a tardar bastante porque va a hacer un request por cada item

___

In [7]:
tipo_cerveza = []
for link in links:
    respuesta = requests.get(link)
    respuesta.encoding = 'utf-8'
    html_2 = respuesta.text
    dom_2 = BS(html_2, features = 'html.parser')
    try:
        tipo_cerveza.append(dom_2.find(class_ = 'especifiacion-box').ul.li.nextSibling.nextSibling.find(class_ = 'info').text.strip())
    except:
        tipo_cerveza.append('Null')

___

Cerramos el browser de Selenium y exportamos la informacion a un CSV

___

In [9]:
driver.quit()

with open('cervezas.csv', 'w', encoding='utf-8') as out:
    out.write('ID, DESCRIPCION, MARCA, PRECIO REG, PRECIO PROMO, DESC PROMO, PRECIO/LT, TIPO PRODUCTO\n')
    for i in range(len(descripcion)):
        out.write(f'{i+1},{descripcion[i]},{marca[i]},{precio_reg[i]},{promo[i]},{desc_promo[i]},{precio_lt[i]},{tipo_cerveza[i]}\n')